In [2]:
import pickle 
import pandas as pd 
from datetime import datetime
df = pd.read_hdf('../../data/processed/train/aggr_train_zeros_25feat.hdf')
df_train_pca = pd.read_hdf('../../data/processed/train/pca.hdf')
threshold = datetime(2015, 9, 1)

df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-1')
pca_cols = ['pca_{}'.format(i) for i in range(4)]
df[pca_cols]= df_train_pca[pca_cols]

In [3]:
pca_cols = ['pca_{}'.format(i) for i in range(4)]

In [12]:
df_train = df[df['date'] < threshold]
#df_train = df
df_X_train, df_y_train = df_train.drop(['item_cnt_day', 'date'], axis=1), df_train['item_cnt_day']

In [13]:
import xgboost as xgb
params = {'max_depth': 13, 'eta': 0.05, 'silent': 1}
num_round = 230
dtrain = xgb.DMatrix(df_X_train, df_y_train.clip(0, 20))
model = xgb.train(params, dtrain, num_round)

In [1]:
import pickle 
import pandas as pd 
from datetime import datetime

In [2]:
model = pickle.load(open('../../models/model_xgb_25_feat_11_PCA.pck', 'rb'))

In [10]:
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from math import sqrt
def get_preds_trained_on_all(model, df_X):
    return model.predict(xgb.DMatrix(df_X)).clip(0, 20)


def previous_value_preds(df_X):
    df_X = df_X.reindex(columns=df_X_train.columns, fill_value=0)
    return df_X['count_aggr_1_month_ago_shop_id_item_id'].clip(0, 20)


def validate(model, df_y_valid, df_X_valid):
    df_y_valid = df_y_valid.copy().clip(0, 20)
    df_y_valid_preds = get_preds_trained_on_all(model, df_X_valid)
    # df_y_valid_preds = previous_value_preds(df_X_valid)
    print('r2 score: ', r2_score(df_y_valid, df_y_valid_preds))
    print('RMSE: ', sqrt(mean_squared_error(df_y_valid, df_y_valid_preds)))

In [6]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test_25feat.hdf')
df_test_pca = pd.read_hdf('../../data/processed/test/pca.hdf')
df_test[pca_cols] = df_test_pca[pca_cols]
df_X_test = df_test.drop(['ID'], axis=1)
df_test['xgb_pca_preds'] = get_preds_trained_on_all(model, df_X_test)
df_test[['ID', 'xgb_pca_preds']].to_hdf('../../data/processed/model_valid/xgb_25_feat_11_PCA_test.hdf', 'xgb_pca_test')

In [14]:
df_valid = df[df['date'] >= threshold]

In [16]:
df_X_valid, df_y_valid = df_valid.drop(['item_cnt_day', 'date'], axis=1), df_valid['item_cnt_day']
validate(model, df_y_valid, df_X_valid)

r2 score:  0.36302502378252866
RMSE:  0.8802481645924013


In [13]:
df_X_valid['preds'] = get_preds_trained_on_all(model, df_X_valid)
a = df_X_valid['preds']
a.to_hdf('../../data/processed/model_valid/xgb_25_feat_11_PCA_valid.hdf', 'xgb_25_feat_PCA')

In [17]:
df_X_valid['preds'] = get_preds_trained_on_all(model, df_X_valid)
a = df_X_valid['preds']
a.to_hdf('../../data/processed/model_valid/pre_valid_train_clip/xgb_25_feat_11_PCA_valid.hdf', 'xgb_25_feat_PCA')

In [11]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test_25feat.hdf')
df_test_pca = pd.read_hdf('../../data/processed/test/pca.hdf')
df_test[pca_cols] = df_test_pca[pca_cols]
df_X_test = df_test.drop(['ID'], axis=1)
df_test['xgb_pca_preds'] = get_preds_trained_on_all(model, df_X_test)
df_test[['ID', 'xgb_pca_preds']].to_hdf('../../data/processed/model_valid/all_train_clip/xgb_25_feat_11_PCA_test.hdf', 'xgb_pca_test')